# Generate lightkurve test data with scope

M. Gully-Santiago  
June 24, 2019
Kepler/K2 GO Office

This notebook generates example test data for evaluating lightkurve's detrending methods.  
Aimed at [Lightkurve GitHub Issue 531](https://github.com/KeplerGO/lightkurve/issues/531).

In [ ]:
import scope

In [ ]:
import lightkurve as lk

In [ ]:
# %load /Users/obsidian/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from astropy.io import fits

## Stellar lightcurves

In [ ]:
target = scope.generate_target()

In [ ]:
target

In [ ]:
plt.plot(target.xpos, target.ypos, '.')

Neat

In [ ]:
import copy

# Simulate test data:

Let's start with the sine wave

> synthetic-k2-flat.targ.fits shall use a flat light curve;

> synthetic-k2-sinusoid.targ.fits shall use a low-frequency sinusoid (e.g. 5-day period);

> synthetic-k2-planet.targ.fits shall use a short-period Neptune-sized planet (e.g. Kp = 14; planet period = 5 days; transit depth = 2000ppm).

In [ ]:
target_sine = copy.copy(target)

In [ ]:
target_sine

In [ ]:
rot_period = 5.0 # days
lc_amp = 0.001 # percent amplitude
injected_sine_lc = 1.0+ lc_amp * np.sin(2.0*np.pi*target.t/rot_period)

In [ ]:
plt.plot(target.t, injected_sine_lc);

In [ ]:
target_sine.add_variability(custom_variability=injected_sine_lc)

In [ ]:
target_sine.plot()

Yay, looks good

In [ ]:
sine_tpf = target_sine.to_lightkurve_tpf(target_id="simulated_sine_P5days_Amp0p001")

### Synthetic TPFs lack requisite metadata found in genuine TPFs

We want to generate synthetic TPFs that satisfy three sometimes-competing demands:
- resemble genuine TPFs in every important way
- possess the least amount of placeholder metadata so as not to confuse synthetic with real data.
- possess the pristine injected signal vector and affiliated metadata


Let's **manually** add these parcels of metadata:
 - row
 - column

In [ ]:
genuine_tpf = lk.search_targetpixelfile(205998445, mission='K2').download()

In [ ]:
genuine_tpf.plot(aperture_mask=genuine_tpf.pipeline_mask)

Compare the genuine TPF header to the synthetic TPF header.

In [ ]:
col = genuine_tpf.get_keyword('1CRV5P', hdu=1, default=0)
row = genuine_tpf.get_keyword('2CRV5P', hdu=1, default=0)

sine_tpf.hdu[1].header.set('1CRV5P', value=col)
sine_tpf.hdu[1].header.set('2CRV5P', value=row)

### Package the pristine signal into the fits file somehow

In [ ]:
sine_tpf.hdu.info()

In [ ]:
extra_hdu = fits.BinTableHDU.from_columns([fits.Column(name='NOISELESS_INPUT', format='E', 
                                 array=injected_sine_lc)])

In [ ]:
extra_hdu.name = 'SIMULATED_SIGNAL'

In [ ]:
sine_tpf.hdu.append(extra_hdu)

In [ ]:
sine_tpf.hdu.info()

Nice!  Let's add some metadata into the header to label the sine period and amplitude.

In [ ]:
sine_tpf.hdu[3].header.set('PERIOD', value=rot_period, comment='Period of noiseless input sine wave')
sine_tpf.hdu[3].header.set('SINE_AMP', value=lc_amp, comment='Amplitude of noiseless input sine wave')

In [ ]:
hdr = sine_tpf.hdu[3].header

In [ ]:
hdr

In [ ]:
sine_tpf.to_fits(output_fn='synthetic-k2-sinusoid.targ.fits', overwrite=True)

tpf_roundtrip = lk.KeplerTargetPixelFile('synthetic-k2-sinusoid.targ.fits')

tpf_roundtrip.plot()

In [ ]:
tpf_roundtrip.hdu.info()

In [ ]:
tpf_roundtrip.hdu['SIMULATED_SIGNAL'].columns

In [ ]:
lc = tpf_roundtrip.to_lightcurve()

lc.plot()

simulated = tpf_roundtrip.hdu['SIMULATED_SIGNAL'].data['NOISELESS_INPUT']
plt.plot(tpf_roundtrip.time, simulated, label='Pristine signal')
plt.legend()

Works well!

In [ ]:
! ls

In [ ]:
pg = tpf_roundtrip.to_lightcurve().to_periodogram()

In [ ]:
pg.plot(view='period')
plt.xlim(0,10)

In [ ]:
tpf_roundtrip.hdu[3].header

Looks like it finds the right period even without detrending.  Nice!

# Simulate a planet transit signal

> synthetic-k2-planet.targ.fits shall use a short-period Neptune-sized planet (e.g. Kp = 14; planet period = 5 days; transit depth = 2000ppm).

We'll need `Starry` for this task.

In [ ]:
import starry

In [ ]:
star = starry.kepler.Primary()

Add limb darkening

In [ ]:
star[1] = 0.40
star[2] = 0.26

In [ ]:
planet = starry.kepler.Secondary(lmax=5)

In [ ]:
planet.r = 0.044 # Rp/R_star, reverse-engineering a nearly 2000 ppm transit without limb darkening
planet.porb = 5 # 5 day period
planet.a = 12.3108 # Semi-major axis, assuming 1 solar mass star

$P^2 \propto A^3$

In [ ]:
AUs = (5.0 * 1.0/365.0)**(2/3) * (1/0.00465047)

In [ ]:
rprs = np.sqrt(2000/1.0e6)

In [ ]:
rprs

In [ ]:
system = starry.kepler.System(star, planet)

In [ ]:
time = target.t
%time system.compute(time)

In [ ]:
plt.subplots(1, figsize=(14, 3))
plt.xlabel('Time [days]', fontsize=16)
plt.ylabel('System Flux', fontsize=16)
plt.plot(time, system.lightcurve);

Nice, let's add this lightcurve to a scope model.

In [ ]:
target_planet = copy.copy(target)

In [ ]:
target_planet.add_variability(custom_variability=system.lightcurve)

In [ ]:
target_planet.plot()

In [ ]:
plan_tpf = target_planet.to_lightkurve_tpf(target_id="simulated_planet_P5days")
plan_tpf.hdu[1].header.set('1CRV5P', value=col)
plan_tpf.hdu[1].header.set('2CRV5P', value=row)

extra_hdu = fits.BinTableHDU.from_columns([fits.Column(name='NOISELESS_INPUT', format='E', 
                                 array=system.lightcurve)])
extra_hdu.name = 'SIMULATED_SIGNAL'
plan_tpf.hdu.append(extra_hdu)
plan_tpf.hdu[3].header.set('PERIOD', value=5.0, comment='Period of noiseless input transit')

plan_tpf.to_fits(output_fn='synthetic-k2-planet.targ.fits', overwrite=True)

In [ ]:
tpf_roundtrip = lk.KeplerTargetPixelFile('synthetic-k2-planet.targ.fits')

tpf_roundtrip.plot()

In [ ]:
tpf_roundtrip.to_lightcurve().plot()
plt.plot(time, system.lightcurve, lw=2, alpha=0.5);

Great!

## Save a default flat, featureless lightcurve

In [ ]:
flat_tpf = target.to_lightkurve_tpf(target_id="simulated_flat_star")
flat_tpf.hdu[1].header.set('1CRV5P', value=col)
flat_tpf.hdu[1].header.set('2CRV5P', value=row)

extra_hdu = fits.BinTableHDU.from_columns([fits.Column(name='NOISELESS_INPUT', format='E', 
                                 array=np.ones(1000))])
extra_hdu.name = 'SIMULATED_SIGNAL'
flat_tpf.hdu.append(extra_hdu)

flat_tpf.to_fits(output_fn='synthetic-k2-flat.targ.fits', overwrite=True)

## The end!